In [1]:
from google.colab import drive

In [2]:
drive.mount("/content/drive", force_remount = True)

Mounted at /content/drive


In [3]:
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -q pyfluidsynth

In [4]:
!git clone https://github.com/magenta/magenta.git

fatal: destination path 'magenta' already exists and is not an empty directory.


In [5]:
cd /content/magenta

/content/magenta


In [6]:
!pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/magenta
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for magenta: filename=magenta-2.1.4-py3-none-any.whl size=19943432 sha256=07ef82878f6ae523d53556937c79dc8c2a8c676a6a56f5b55837f31882a098a0
  Stored in directory: /tmp/pip-ephem-wheel-cache-wot2y_xi/wheels/50/19/f2/0b8436a5062448cd148094ee41072a2690ae84b02d63ddeb54
Successfully built magenta
  Attempting uninstall: magenta
    Found existing installation: magenta 2.1.4
    Uninstalling magenta-2.1.4:
      Successfully uninstalled magenta-2.1.4


In [18]:
from google.colab import files
import magenta.music as mm
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
import numpy as np
import os
import tensorflow as tf
import note_seq
from note_seq import midi_synth

# Necessary until pyfluidsynth is updated (>1.2.5).
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [21]:
dataset_url = "https://storage.googleapis.com/magentadata/datasets/groove/groove-v1.0.0-midionly.zip"
tf.keras.utils.get_file(origin=dataset_url, 
                        fname='/content/drive/MyDrive/poza_data/groove_data.zip',
                        extract=True)

3260318/3260318 [==============================] - 0s 0us/step


In [23]:
import zipfile

In [24]:
with zipfile.ZipFile("/content/drive/MyDrive/poza_data/groove_data.zip", "r") as f:
  f.extractall("/content/drive/MyDrive/poza_data/groove_data")

In [26]:
!convert_dir_to_note_sequences\
  --input_dir=/content/drive/MyDrive/poza_data/groove_data/groove\
  --output_file=/content/drive/MyDrive/poza/data/sequences.tfrecord\
  --recursive

/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB thr

In [8]:
cd magenta/models/music_vae/

/content/magenta/magenta/models/music_vae


In [10]:
!music_vae_train \
--config=groovae_4bar \
--run_dir=/content/drive/MyDrive/poza/tmp/music_vae/ \
--mode=train \
--num_steps=20000 \
--examples_path=/content/drive/MyDrive/poza/data/sequences.tfrecord

/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB thr

In [16]:
def play(note_sequence):
  note_seq.play_sequence(start_notes_at_0(note_sequence), synth=note_seq.fluidsynth)

def download(note_sequence, filename):
  note_seq.sequence_proto_to_midi_file(note_sequence, filename)
  files.download(filename)

def start_notes_at_0(s):
  for n in s.notes:
    if n.start_time < 0:
      n.end_time -= n.start_time
      n.start_time = 0
  return s

In [10]:
music_vae = TrainedModel(
      configs.CONFIG_MAP["groovae_4bar"], 
      batch_size=512, 
      checkpoint_dir_or_path="/content/drive/MyDrive/poza/tmp/music_vae/train/model.ckpt-20000")

/content/magenta/magenta/models/music_vae/lstm_utils.py:99: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  name=name),
/content/magenta/magenta/contrib/rnn.py:750: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._names["W"], [input_size + self._num_units, self._num_units * 4])
/content/magenta/magenta/contrib/rnn.py:753: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  initializer=tf.constant_initializer(0.0))
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
/content/magenta/magenta/models/music

In [22]:
samples = music_vae.sample(3, temperature=1,length=64)
for idx, s in enumerate(samples):
  play(s)
  download(s, f"test{idx}.midi")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>